In [2]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import axes
import random
%matplotlib inline

In [ ]:
import networkx as nx
import random
import sys 


G = nx.read_adjlist("p2p-Gnutella08-mod.txt", create_using=nx.DiGraph())


nodes = list(G.nodes())
visit_counts = {node: 0 for node in nodes} # at first every node has 0 visits 

m = 0.15
steps = 5000000

current_node = random.choice(nodes) # pick a random inital node

for _ in range(steps): # start surfing 
    visit_counts[current_node] += 1  # count visit for the current node
    p = random.random() # for each step generate a random probability of surfing to another page

    if p < m: # if this probability is less that the damping factor m jump to a random node 
        current_node = random.choice(nodes)
    else: # but if the probability is equal or greater than m randomly follow a link or at least attempt to do so (dangling nodes exist)
        successors = list(G.successors(current_node))
        if successors: # if there are outgoing links to other nodes, pick randomly from them
            current_node = random.choice(successors)
        else: # if no outgoing edges (dangling node), jump to a random node
            current_node = random.choice(nodes)
    


# printing the 10 most visited by sorting by the values in the dictionary in descending order
# then slicing for top 10
top10_sorted_visits = sorted(visit_counts.items(), key=lambda item: item[1], reverse=True)[:10]

for rank, (node, visits) in enumerate(top10_sorted_visits, start = 1):
    print(f"#{rank}: Node {node} - Visits: {visits}")

KeyboardInterrupt: 

In [ ]:
import networkx as nx
import numpy as np

iterations = 100 

G = nx.read_adjlist("p2p-Gnutella08-mod.txt", create_using=nx.DiGraph())

m = 0.15 # damping factor
n = len(G) # number of nodes

x_k = np.ones(n) / n # initial rank vector - an array of 1s with the size of n
# here we are not creating the whole matrix A (n by n) for optimization purposes

mSx_k = np.full(n, m/n) # an array of size n filled with value m/n, responsible for random jumping
# again not creating the whole matrix since all its values are the same 

# append the node to the dangling nodes list if it has no outlinks to sucessors (its out_degree is 0)
dangling_nodes = [node for node in G.nodes() if G.out_degree(node) == 0] 

print(f"Dangling nodes: {dangling_nodes} (Total: {len(dangling_nodes)})")

# creating this dictionary with nodes as keys and their indexes as values and filling it up as nodes in G appear
# so we can correctly perform operations with them
node_index = {node: i for i, node in enumerate(G.nodes())}

for iteration in range(iterations):
    x_k_1 = np.zeros(n)
    D_x_k = (1 - m) * np.sum([x_k[node_index[node]] for node in dangling_nodes]) / n 
    # we compute the constant D_x_k once for every iteration 
    for node in G.nodes():
        for predecessor in G.predecessors(node):
            outlinks_predecessor = G.out_degree(predecessor)
            weight_from_predecessor = (1 - m) * x_k[node_index[predecessor]] / outlinks_predecessor
            x_k_1[node_index[node]] += weight_from_predecessor
            # to each element in the x_k_1 array representing the importance score of the node
            # add weight from the predecessor
    
    # to get the true importance score we sum the weights from predecessors, random jumping and dangling nodes
    x_k_1 = x_k_1 + mSx_k + D_x_k

    #check if convergence is reached
    if np.linalg.norm(x_k - x_k_1) < 1e-6:
        print(f"This took {iteration + 1} iterations.")
        break
    

    # convergence has not been reached yet so we update x_k for the next iteration
    # eigenvalue lambda for pagerank is 1 so Av = v so in our case we assign x_k_1 to become x_k and new iteration begins
    x_k = x_k_1 

print(f'Importance vector after {iteration + 1} iterations is {x_k}')

x_k_dict = {node: float(x_k[node_index[node]]) for node in G.nodes()}
x_k_dict_sorted = dict(sorted(x_k_dict.items(), key=lambda item: item[1], reverse=True))


top10_nodes = list(x_k_dict_sorted.items())[:10]

for rank, (node, score) in enumerate(top10_nodes, start=1):
    print(f"#{rank}: Node {node} - Score: {score}")






Dangling nodes: ['1', '2', '6', '10', '826', '1097', '1591', '1897', '1898', '1899', '258', '491', '1021', '1418', '1900', '1901', '1902', '1903', '121', '128', '426', '754', '2064', '3002', '520', '1905', '1906', '246', '12', '16', '18', '19', '2074', '809', '1050', '2254', '3169', '3619', '4408', '4953', '368', '1908', '1909', '1702', '1833', '1916', '1918', '1919', '1921', '23', '24', '26', '901', '960', '1511', '1624', '1911', '1912', '1913', '1914', '1915', '1516', '1517', '870', '1219', '1954', '2630', '3637', '3874', '957', '34', '35', '37', '40', '41', '42', '725', '1376', '2002', '2167', '2168', '2169', '2170', '614', '1378', '1925', '1927', '1928', '1604', '1679', '1931', '1933', '1935', '1936', '45', '46', '47', '48', '50', '51', '52', '53', '54', '338', '2366', '2893', '3065', '3593', '3840', '3999', '56', '57', '58', '59', '1937', '1938', '1940', '1944', '1993', '1997', '1064', '1194', '1196', '1959', '1960', '1961', '1962', '1963', '1364', '1977', '1978', '1979', '1980', 

In [5]:
nx.pagerank(G)

{'0': 0.00010061505853980175,
 '1': 0.00010917398605861922,
 '2': 0.00017116880733588808,
 '3': 0.0014181699785875226,
 '4': 0.0015447615491227767,
 '5': 0.0018239540797636209,
 '6': 0.00010917398605861922,
 '7': 0.0014955078138058,
 '8': 0.0012606249868507581,
 '9': 0.001198468540417282,
 '10': 0.0003107572910744247,
 '703': 0.00026635801039895314,
 '826': 0.0002551539465463142,
 '1097': 0.00030588732120622315,
 '1287': 0.00028161737494289,
 '1591': 0.0003929454800259496,
 '1895': 0.00022052084485795086,
 '1896': 0.00029326593824984616,
 '1897': 0.00022052084485795086,
 '1898': 0.00024070955634483194,
 '1899': 0.00023283194121715293,
 '144': 0.0013113363985150358,
 '258': 0.0002530817948549188,
 '491': 0.0003604451785972098,
 '1021': 0.00037049382038905314,
 '1418': 0.0002431647560001845,
 '1669': 0.0002865819898218685,
 '1900': 0.0003321300393876918,
 '1901': 0.00028865695260892017,
 '1902': 0.0002312598507767206,
 '1903': 0.0004772658319815221,
 '121': 0.001206065426187191,
 '127': 